In [39]:
# Declaring the imports
import pandas as pd
import numpy as np
import re

In [40]:
# Paths to the CSV files
TRAIN_PATH = "../data/Training.csv"
TEST_PATH = "../data/Testing.csv"

In [41]:
# Function for loading the CSV files
def load_data(path):
    df = pd.read_csv(path)
    return df

In [42]:
# Detect the target column
def detect_target_column(df):
    candidates = ['Disease', 'Prognosis', 'prognosis', 'disease']
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(f"Could not find target column among { candidates }. Found: { list(df.columns) }")

# Detect the symptom columns
def detect_symptom_columns(df):
    target_column = detect_target_column(df)
    cols = [c for c in df.columns if c != target_column]
    return cols

In [43]:
# Encoding the Symptom labels
def encode_labels(raw_values):
    classes = list(dict.fromkeys(pd.Series(raw_values).astype(str)))
    cls_to_int = { c: i for i, c in enumerate(classes) }
    int_values = np.array([cls_to_int[str(v)] for v in raw_values], dtype = np.int64)
    return int_values, classes, cls_to_int

In [44]:
# Cleaning the User given text input to create the input vector for the model
def clean_symptom_text(user_input):
    user_input = user_input.lower()
    user_input = re.sub(r'[^a-z\s]', '', user_input)
    words = user_input.split()
    return set(words)

In [45]:
# Implementing the one-hot encoding function
def one_hot(y_int, n_classes):
    y = np.zeros((y_int.shape[0], n_classes), dtype = np.float32)
    y[np.arange(y_int.shape[0]), y_int] = 1.0
    return y

In [46]:
# Implementing the User Vector build method
def build_user_vector(user_input_text, symptom_index):
    vector = np.zeros((1, len(symptom_index)), dtype = np.float32)
    if user_input_text:
        user_symptoms = [s.strip().lower() for s in user_input_text.split(",")]
        for s in user_symptoms:
            if s in symptom_index:
                vector[0, symptom_index[s]] = 1.0
    return vector

In [47]:
# Implementing the Dense Neural Network Layer class
class Layer_Dense:
    def __init__(self, n_inputs, n_outputs, w_init):
        if w_init == "he":
            self.w = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0 / n_inputs)
        elif w_init == "xavier":
            self.w = np.random.randn(n_inputs, n_outputs) * np.sqrt(1.0 / n_inputs)
        self.b = np.zeros((1, n_outputs))
        self.dw = np.zeros_like(self.w)
        self.db = np.zeros_like(self.b)
    
    def forward_pass(self, x):
        self.x = x
        return np.dot(x, self.w) + self.b
    
    def backward_pass(self, dz):
        self.dw = np.dot(self.x.T, dz)
        self.db = np.sum(dz, axis = 0, keepdims = True)
        dx = np.dot(dz, self.w.T)
        return dx
    
    def step(self, lr):
        self.w -= lr * self.dw
        self.b -= lr * self.db

In [48]:
# Implementing the ReLU activation function class
class ReLU_Activation:
    def forward_pass(self, z):
        self.z = z
        return np.maximum(0.0, z)
    def backward_pass(self, dvalues_a):
        dz = dvalues_a.copy()
        dz[self.z <= 0.0] = 0
        return dz

In [49]:
# Implementing the Softmax activation function class
class Softmax_Activation:
    def forward_pass(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True))
        self.probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True)

In [50]:
# Implementation of common Loss class
class Loss:
    def calculate_loss(self, outputs, t_values):
        sample_losses = self.forward_pass(outputs, t_values)
        data_loss = np.mean(sample_losses)
        return data_loss

In [51]:
# Implemention Categorical Cross-entropy Loss class
class CategoricalCrossEntropy_Loss:
    def calculate_loss(self, p_values, t_values):
        samples = len(p_values)
        p_values_clipped = np.clip(p_values, 1e-7, 1 - 1e-7)
        if len(t_values.shape) == 1:
            correct_confidences = p_values_clipped[range(samples), t_values]
        elif len(t_values.shape) == 2:
            correct_confidences = np.sum(p_values_clipped * t_values, axis = 1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return np.mean(negative_log_likelihoods)

In [52]:
# Implementing the combined Softmax activation and Cross-entropy loss class
class Softmax_Activation_CategoricalCrossEntropy_Loss:
    def __init__(self):
        self.activation = Softmax_Activation()
        self.loss = CategoricalCrossEntropy_Loss()
    
    def forward_pass(self, inputs, t_values):
        self.activation.forward_pass(inputs)
        self.output = self.activation.probabilities
        return self.loss.calculate_loss(self.output, t_values)
    
    def backward_pass(self, d_values, t_values):
        samples = len(d_values)
        if len(t_values.shape) == 2:
            t_values = np.argmax(t_values, axis = 1)
        self.d_inputs = d_values.copy()
        self.d_inputs[range(samples), t_values] -= 1
        self.d_inputs = self.d_inputs / samples

In [53]:
# Implementing the Dropout class
class Dropout:
    def __init__(self, rate):
        self.rate = rate

    def forward_pass(self, x, training = True):
        if not training:
            return x
        self.mask = np.random.binomial(1, 1 - self.rate, size=x.shape) / (1 - self.rate)
        return x * self.mask
    
    def backward_pass(self, d_values):
        return d_values * self.mask

In [54]:
# Implementing the core MLP class
class MLP:
    def __init__(self, input_dim, hidden1 = 64, hidden2 = 32, n_classes = 10, learning_rate = 0.01, dropout_rate = 0.2):
        self.input_layer = Layer_Dense(input_dim, hidden1, w_init = "he")
        self.activation_1 = ReLU_Activation()
        self.dropout_1 = Dropout(dropout_rate)
        self.hidden_layer_1 = Layer_Dense(hidden1, hidden2, w_init = "he")
        self.activation_2 = ReLU_Activation()
        self.dropout_2 = Dropout(dropout_rate)
        self.hidden_layer_2 = Layer_Dense(hidden2, n_classes, w_init = "xavier")
        self.softmax_loss = Softmax_Activation_CategoricalCrossEntropy_Loss()
        self.learning_rate = learning_rate
    
    def forward(self, x, y_true = None):
        z1 = self.input_layer.forward_pass(x)
        a1 = self.activation_1.forward_pass(z1)
        d_a1 = self.dropout_1.forward_pass(a1, True)
        z2 = self.hidden_layer_1.forward_pass(d_a1)
        a2 = self.activation_2.forward_pass(z2)
        d_a2 = self.dropout_2.forward_pass(a2, True)
        z3 = self.hidden_layer_2.forward_pass(d_a2)
        if y_true is not None:
            loss = self.softmax_loss.forward_pass(z3, y_true)
            return self.softmax_loss.output, loss
        self.softmax_loss.activation.forward_pass(z3)
        return self.softmax_loss.activation.probabilities
    
    def backward(self, y_true):
        self.softmax_loss.backward_pass(self.softmax_loss.output, y_true)
        da2 = self.hidden_layer_2.backward_pass(self.softmax_loss.d_inputs)
        dz2 = self.activation_2.backward_pass(da2)
        d_dz2 = self.dropout_2.backward_pass(dz2)
        da1 = self.hidden_layer_1.backward_pass(d_dz2)
        dz1 = self.activation_1.backward_pass(da1)
        d_dz1 = self.dropout_1.backward_pass(dz1)
        _ = self.input_layer.backward_pass(d_dz1)
    
    def w_step(self):
        for layer in [self.input_layer, self.hidden_layer_1, self.hidden_layer_2]:
            layer.step(self.learning_rate)
    
    def predict(self, x):
        probs = self.forward(x)
        return np.argmax(probs, axis = 1), probs
    
    def fit(self, x, y_one_hot, epochs = 50, batch_size = 64, x_val = None, y_val = None):
        n = x.shape[0]
        for epoch in range(1, epochs + 1):
            idx = np.random.permutation(n)
            x, y_one_hot = x[idx], y_one_hot[idx]
            epoch_loss = 0.0
            seen = 0
            for start in range(0, n, batch_size):
                end = min(start + batch_size, n)
                xb, yb = x[start : end], y_one_hot[start : end]
                probs, loss = self.forward(xb, yb)
                self.backward(yb)
                self.w_step()
                epoch_loss += loss * (end - start)
                seen += (end - start)
            epoch_loss /= seen
            if epoch == 1 or epoch % 5 == 0:
                msg = f"Epoch {epoch:3d} | loss {epoch_loss:.4f}"
                if x_val is not None and y_val is not None:
                    y_pred, _ = self.predict(x_val)
                    val_acc = (y_pred == y_val).mean()
                    msg += f" | val_acc {val_acc:.4f}"
                print(msg)

In [55]:
# Implementing the Conversation and Vector creation class
class Conversation_and_Vector:
    def __init__(self, model, classes, symptom_index, threshold_gap):
        self.model = model
        self.classes = classes
        self.symptom_index = symptom_index
        self.threshold_gap = threshold_gap
        self.vector = np.zeros((1, len(symptom_index)), dtype=np.float32)

    def update_vector(self, user_input):
        tokens = [w.strip().lower() for w in re.split(r"[,.;]", user_input) if w.strip()]
        for t in tokens:
            if t in self.symptom_index:
                self.vector[0, self.symptom_index[t]] = 1.0

    def predict(self):
        _, probs = self.model.predict(self.vector)
        top_index_2 = np.argsort(probs[0])[-2:][::-1]
        top_conf, second_conf = probs[0][top_index_2[0]], probs[0][top_index_2[1]]
        top_disease, second_disease = self.classes[top_index_2[0]], self.classes[top_index_2[1]]
        return top_disease, top_conf, second_disease, second_conf
    
    def conversation_loop(self):
        while True:
            user_input = input().strip().lower()
            if not user_input:
                break
            self.update_vector(user_input)
            top_disease, top_conf, second_disease, second_conf = self.predict()
            # gap = top_conf - second_conf
            # if gap >= self.threshold_gap:
            if top_conf >= 0.9:
                print(f"Final prediction: { top_disease } with confidence: { top_conf }")
                break
            else:
                print(f"It could be { top_disease } confidence: { top_conf } or { second_disease } confidence: { second_conf }. Could you share more symptoms...")

In [56]:
# Implementing the Main method
def main():
    train_df = load_data(TRAIN_PATH)
    test_df = load_data(TEST_PATH)

    if "Unnamed: 133" in train_df.columns:
        train_df.drop('Unnamed: 133', axis=1, inplace=True)
    if "Unnamed: 133" in test_df.columns:
        test_df.drop('Unnamed: 133', axis=1, inplace=True)

    target_column = detect_target_column(train_df)
    symptom_columns = detect_symptom_columns(train_df)
    
    x_train = train_df[symptom_columns].values.astype(np.float32)
    y_train_int, classes, cls_to_int = encode_labels(train_df[target_column].astype(str).values)
    y_train_one_hot = one_hot(y_train_int, len(classes))

    x_test = test_df[symptom_columns].values.astype(np.float32)
    y_test_int = np.array([cls_to_int[str(v)] for v in test_df[target_column].astype(str).values], dtype = np.int64)
    y_test_one_hot = one_hot(y_test_int, len(classes))

    # print(f"Shape of x_test before fit: {x_test.shape}")
    # print(f"Shape of y_test_int before fit: {y_test_int.shape}")

    model = MLP(x_train.shape[1], 64, 32, len(classes), 0.01)
    model.fit(x_train, y_train_one_hot, 100, 64, x_test, y_test_int)

    y_pred, _ = model.predict(x_test)
    print(f"\nTest accuracy: {(y_pred == y_test_int).mean():.4f}")

    symptom_index = {s.lower(): i for i, s in enumerate(symptom_columns)}
    print("\nEnter symptoms separated by commas (empty line to exit):")
    # while True:
    #     user_text = input("\nYour symptoms: ").strip()
    #     if not user_text:
    #         break
    #     x_vec = build_user_vector(user_text, symptom_index)
    #     pred_idx, probs = model.predict(x_vec)
        # print(f"=> Predicted disease: {classes[pred_idx[0]]}")
    conversation_and_vector = Conversation_and_Vector(model, classes, symptom_index, 0.1)
    conversation_and_vector.conversation_loop()

In [57]:
# Calling the Main method
main()

Epoch   1 | loss 3.7469 | val_acc 0.0238
Epoch   5 | loss 3.4834 | val_acc 0.0952
Epoch  10 | loss 3.0750 | val_acc 0.2619
Epoch  15 | loss 2.5171 | val_acc 0.4524
Epoch  20 | loss 1.9261 | val_acc 0.6667
Epoch  25 | loss 1.4164 | val_acc 0.7143
Epoch  30 | loss 1.0700 | val_acc 0.8810
Epoch  35 | loss 0.7903 | val_acc 0.8810
Epoch  40 | loss 0.6377 | val_acc 0.8333
Epoch  45 | loss 0.5018 | val_acc 0.8571
Epoch  50 | loss 0.4197 | val_acc 0.9048
Epoch  55 | loss 0.3600 | val_acc 0.9762
Epoch  60 | loss 0.3086 | val_acc 0.9524
Epoch  65 | loss 0.2870 | val_acc 0.9524
Epoch  70 | loss 0.2455 | val_acc 0.9524
Epoch  75 | loss 0.2203 | val_acc 0.9524
Epoch  80 | loss 0.2083 | val_acc 0.9524
Epoch  85 | loss 0.1936 | val_acc 0.9524
Epoch  90 | loss 0.1767 | val_acc 0.9048
Epoch  95 | loss 0.1707 | val_acc 0.9524
Epoch 100 | loss 0.1601 | val_acc 0.9286

Test accuracy: 0.9762

Enter symptoms separated by commas (empty line to exit):
It could be Drug Reaction confidence: 0.7226851069891418 o